# **Recitation 7**

In [ ]:
# Click the "Play" button on the left to import packages.
%pip install -i https://pypi.gurobi.com gurobipy;
import gurobipy
import cvxpy as cp
import numpy as np
import pandas as pd

# Below function will simplify the conversion of numpy arrays into dataframes
def to_df(a): # turns array into pandas dataframe
  df = pd.DataFrame(data    = a[1:,1:] , # values
                    index   = a[1:,0]  , # 1st column as rows
                    columns = a[0,1:]  ) # 1st row as the column names
  df = df.astype('float') # Turn dataframe entries into floats
  return df

Looking in indexes: https://pypi.gurobi.com


**Reference Code**



In [ ]:
# Define supply and demand dictionaries
supply = {'Seattle'   : 350,
          'San Diego' : 600}

demand = {'New York'  : 325,
          'Chicago'   : 300,
          'Topeka'    : 275,
          'Unused'    : 50}

# Define the cost of transportation
cost = to_df(np.array([[ '',          'New York',   'Chicago',  'Topeka', 'Unused'],
                       ['Seattle' ,   225,          153,        162,      0],
                       ['San Diego',  225,          162,        126,      0]]))

In [ ]:
# Define function to solve transportation problem
def transport(supply, demand, cost):

  # Enforce all non-negative supply and demand
  assert all((val >= 0) for val in supply.values())
  assert all((val >= 0) for val in demand.values())

  # Check supply and demand match
  assert sum(supply.values()) == sum(demand.values())

  # Check there is cost matches supply-demand
  assert set(cost.index) == set(supply.keys())
  assert set(cost.columns) == set(demand.keys())

  # Check all costs are numeric and nonnegative
  assert np.all(cost.to_numpy() >= 0)

  # Create transportation variables
  trans = {}
  for orig in supply.keys():
    for dest in demand.keys():
      trans[orig, dest] = cp.Variable(nonneg = True) # Units to be shipped

  # Define objective
  total_cost = sum(cost.loc[orig,dest] * trans[orig,dest] for orig in supply.keys() for dest in demand.keys())

  # Create constraints
  constraints = []
  for orig in supply.keys():
    constraints += [ sum(trans[orig,dest] for dest in demand.keys()) == supply[orig] ]
  for dest in demand.keys():
    constraints += [ sum(trans[orig,dest] for orig in supply.keys()) == demand[dest]  ]

  # Solve problem
  prob = cp.Problem(cp.Minimize(total_cost), constraints)
  prob.solve(solver = cp.GUROBI)

  # Print objective
  print('objective = %s' % prob.value)

  # Print transportation
  print('trans = ')
  output = pd.DataFrame(index=cost.index, columns=cost.columns, dtype=float)
  for orig in supply.keys():
    for dest in demand.keys():
      output.loc[orig, dest] = trans[orig,dest].value
  print(output)

  # ADD CODE FOR (a.1) BELOW:

In [ ]:
# Solve transportation problem
transport(supply, demand, cost)

**Exercise**

A department needs to assign 11 people to an equal number of offices. Relative to the transporation problem, one can consider the origins now as individual people, and the destinations as individual offices.

Each person is
assigned one office, and each office is occupied by one person.
Thus, all of the parameter values *supply[i]* and *demand[j]* are 1.

In the transportation problem, there is a 2D table of *cost* values: In the office-assignment context, *cost[i,j]* corresponds to the *ranking* of person *i* for office *j*. Smaller numbers like "1" (i.e. first-choice) indicate more preferable, while larger numbers like "8" (i.e. eighth-choice) indicate less preferable.

Recall that decision variables in the transportation problem are a 2D table that we will call *Trans*. We interpret *Trans[i,j]* as the "amount" of person *i* that is assigned to office *j*. Thus, if *Trans[i,j]* is 1, then person *i* will occupy office *j*. Similarly, if *Trans[i,j]* is 0, then person *i* will not occupy office *j*. Each person *i* ranked the offices, so for the objective we wish to minimize the sum of *cost[i,j]* ** Trans[i,j]*, which is the sum of the rankings of the office each person is assigned to. By minimizing this sum, we aim to maximize people's happiness with the office assignments. The cost table is below:

\begin{array}{cccccccccccc}
& \text{C118} &\text{C138} &\text{C140} &\text{C246} &\text{C250} &\text{C251} & \text{D237} & \text{D239} & \text{D241} & \text{M233} & \text{M239}\\
\text{Coullard} & 6& 9& 8& 7& 11& 10 & 4 & 5 & 3 & 2 & 1\\
\text{Daskin} & 11& 8& 7& 6& 9& 10 & 1 & 5 & 4 & 2 & 3\\
\text{Hazen} & 9& 10& 11& 1& 5& 6 & 2 & 7 & 8 & 3 & 4\\
\text{Hopp} & 11& 9& 8& 10& 6& 5 & 1 & 7 & 4 & 2 & 3\\
\text{Iravani} & 3& 2& 8& 9& 10& 11 & 1 & 5 & 4 & 6 & 7\\
\text{Linetsky} & 11& 9& 10& 5& 3& 4 & 6 & 7 & 8 & 1 & 2\\
\text{Mehrotra} & 6& 11& 10& 9& 8& 7 & 1 & 2 & 5 & 4 & 3\\
\text{Nelson} & 11& 5& 4& 6& 7& 8 & 1 & 9 & 10 & 2 & 3\\
\text{Smilowitz} & 11& 9& 10& 8& 6& 5 & 7 & 3 & 4 & 1 & 2\\
\text{Tamhane} & 5& 6& 9& 8& 4& 3 & 7 & 10 & 11 & 2 & 1\\
\text{White} & 11& 9& 8& 4& 6& 5 & 3 & 10 & 7 & 2 & 1\\
\end{array}

**(a)**

**(a.1)**
Modify the reference code to also print the final cumulative cost of transportation between each origin-destination pair (i.e. the cost multiplied by the amount transported).

**(a.2)**
Using the description above, map the office assignment problem to the transportation problem. Create new data variables for the supply (Python dictionary), demand (Python dictionary), and cost (Pandas Dataframe) based on this mapping. Name your created data `supply_a`, `demand_a`, and `cost_a`. Pass this new data to the `transport` function to solve the office-assignment problem.

In [ ]:
## ADD PART (a) CODE BELOW ##

In [ ]:
 # After writing the data for part (a), run the function below
 transport(supply_a, demand_a, cost_a)

**(b)**
An assignment that gives even one person a very low-ranked office may be unacceptable, even if the total of the rankings is optimized. In particular, our solution gives one individual her sixth choice; to rule this out, change all preferences of six or larger in the cost data to 99, so that they will become very unattractive. Create a new cost dataframe (call it `cost_b`) with the changes and resolve the assignment problem again.

In [ ]:
## ADD PART (b) CODE BELOW ##

In [ ]:
 # After writing the data for part (b), run the function below
 transport(supply_a, demand_a, cost_b)

**(c)** Repeat part (b) but replace all entries *five* or larger in a new function called `cost_c`. What do you find?

In [ ]:
## ADD PART (c) CODE BELOW ##

In [ ]:
 # After writing the data for part (c), run the function below
 transport(supply_a, demand_a, cost_c)

**(d)**
Suppose now that offices C118, C250, and C251 become unavailable, and you have to put two people each into C138, C140, and C246. Add 20 to each ranking for these three offices, to reflect
the fact that anyone would prefer a private office to a shared one. What other modifications to the
model and data would be necessary to handle this situation? What optimal assignment do you get?

Create a new model function `transport_d` as well as new data named `supply_d`, `demand_d`, and `cost_d` to reflect this situation.

Create a new dictionary object called `share_d` that encodes, for each office, whether it can be shared (encoded as 1) or not shared (encoded as 0). Solve this new problem. For example, the dictionary

```
share_example = {'C118' : 0,
                 'C138' : 1}
```
encodes that office C118 can not be shared, but C138 can. The `share_d` would also need to be passed to `transport_d`.

Hint: Start by copying the reference model as well as the supply, demand, and cost from part (a). Change only what you need to for reflecting the new situation. Some of these variables may not need to be changed at all.

In [ ]:
## ADD DATA CODE FOR PART (d) BELOW ##

In [ ]:
# Define function to solve transportation problem
def transport_d(supply, demand, cost, share):

  # ADD MODEL CODE FOR PART (d) BELOW #

In [ ]:
 # After writing the data for part (d), run the function below
 transport_d(supply_d, demand_d, cost_d, share_d)

**(e)** Some people may have seniority that entitles them to greater consideration in their choice of office. Explain how you could enhance the model to use seniority level data for each person.

Write a new model function `transport_e` that reflects this situation. You do not have to create data and optimize anything, but your code should be understandable.

Hint: You need to do something similar to part (d).

In [ ]:
# Define function to solve transportation problem
def transport_e(supply, demand, cost, seniority):

  # ADD MODEL CODE FOR PART (e) BELOW #